# Replicate Quickstart

In this quickstart you will learn how to use Replicate as a feedback function provider. [Replicate](https://replicate.com/docs) makes it easy to run machine learning models in the cloud from your own code. You can run open-source models, or run your own public and private models.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/replicate_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need a Replicate key.

In [ ]:
import os
os.environ["REPLICATE_API_TOKEN"] = "..."

### Import from LangChain and TruLens

In [ ]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru, Replicate as fReplicate
tru = Tru()
tru.reset_database()

### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [ ]:
import replicate

def _handle_streaming(output) -> str:
        full_output = []
        for item in output:
            print(item, end="")
            full_output.append(item)
        # Join the list of strings into a single string
        output_str = ''.join(full_output)
        # Replace single quotes with double quotes
        output_str = output_str.replace("'", '"')
        return output_str  # Add this line

def llm_standalone(prompt):
    output = replicate.run(
    "mistralai/mistral-7b-instruct-v0.1:83b6a56e7c828e667f21fd596c338fd4f0039b46bcfa18d973e8e70e455fda70",
    input={"prompt": prompt}
    )
    return _handle_streaming(output)

### Send your first request

In [ ]:
prompt_input = 'What is a good name for a store that sells colorful socks?'

In [ ]:
llm_response = llm_standalone(prompt_input)

display(llm_response)

## Initialize Feedback Function(s)

In [ ]:
# Initialize Replicate-based feedback function collection class:
freplicate = fReplicate()
# Define a relevance function using LiteLLM
relevance = Feedback(freplicate.relevance).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

## Instrument chain for logging with TruLens

In [ ]:
from trulens_eval import TruBasicApp
tru_recorder = TruBasicApp(llm_standalone,
    app_id='Replicate_app',
    feedbacks=[relevance])

In [ ]:
with tru_recorder as recording:
    llm_response = tru_recorder.app(prompt_input)

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0]

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all